In [84]:
import os
import json
import pandas as pd
os.chdir("/home/t-iswatts/Desktop/MultilingualBlanketEval")
os.getcwd()

'/home/t-iswatts/Desktop/MultilingualBlanketEval'

In [85]:
final_output = []

datasets = os.listdir("results")
ignore = ["IN22-Gen", "XLSum", "xrisawoz"]
models = set()
for dataset in datasets:
    if dataset in ignore:
            continue
    for model in os.listdir(os.path.join("results/", dataset)):
        models.add(model)

print(models)

for model in models:
    dataset_name = []
    languages = []
    results = []
    metrics = []
    strategies = []
    for dataset in datasets:
        if dataset in ignore:
            continue
        
        try:
            for language in os.listdir(os.path.join("results/", dataset, model)):
                for strategy in os.listdir(os.path.join("results/", dataset, model, language)):
                    with open(os.path.join("results/", dataset, model, language, strategy, "results.json"), "r") as f:
                        result = json.load(f)
                    
                    for key in list(result["metrics"].keys())[:2]:
                        dataset_name.append(dataset)
                        languages.append(language)
                        metrics.append(key)
                        results.append(result["metrics"][key])
                        strategies.append(strategy)
                        
                    
        except Exception as e:
            print(e)
            # print(f"Model: {model} not Evaluated on Dataset: {dataset}")
            pass
        
    final_output.append(pd.DataFrame({"Dataset": dataset_name, "Language": languages, "Metric": metrics, model: results, "Strategy": strategies}))
        # except:
        #     print(f"Model: {model} not Evaluated on Dataset: {dataset}")

{'gemma-2b-it', 'gemma-7b-it', 'gemini-pro'}
[Errno 2] No such file or directory: 'results/panx/gemma-2b-it'
[Errno 2] No such file or directory: 'results/gluecos_sentiment/gemma-2b-it'
[Errno 2] No such file or directory: 'results/gluecos_nli/gemma-2b-it'
[Errno 2] No such file or directory: 'results/pawsx/gemma-2b-it/ko/PivotLang_ko_PromptName_PAWS-ANLI GPT3_FewShotK_8/results.json'
[Errno 2] No such file or directory: 'results/panx/gemma-7b-it'
[Errno 2] No such file or directory: 'results/gluecos_sentiment/gemma-7b-it'
[Errno 2] No such file or directory: 'results/gluecos_nli/gemma-7b-it'
[Errno 2] No such file or directory: 'results/xcopa/gemma-7b-it/en/PivotLang_en_PromptName_plausible_alternatives_discrete_FewShotK_8/results.json'
[Errno 2] No such file or directory: 'results/xstory_cloze/gemma-7b-it/ar/PivotLang_ar_PromptName_Answer Given options_Verbalizer_identity_FewShotK_8wthInstruction/results.json'
[Errno 2] No such file or directory: 'results/afriqa/gemma-7b-it/fon/Pivot

In [86]:
len(final_output)

3

In [87]:
final_output[0]

,Dataset,Language,Metric,gemma-2b-it,Strategy
0,xquad,es,exact_match,3.361345,PivotLang_en_PromptName_answer_given_context_a...
1,xquad,es,f1,19.659572,PivotLang_en_PromptName_answer_given_context_a...
2,xquad,zh,exact_match,4.453782,PivotLang_en_PromptName_answer_given_context_a...
3,xquad,zh,f1,12.519405,PivotLang_en_PromptName_answer_given_context_a...
4,xquad,el,exact_match,6.974790,PivotLang_en_PromptName_answer_given_context_a...
...,...,...,...,...,...
178,tydiqa,en,f1,51.855510,PivotLang_en_PromptName_answer_given_context_a...
179,tydiqa,bn,exact_match,4.424779,PivotLang_bn_PromptName_answer_given_context_a...
180,tydiqa,bn,f1,20.504604,PivotLang_bn_PromptName_answer_given_context_a...
181,tydiqa,sw,exact_match,44.689379,PivotLang_sw_PromptName_answer_given_context_a...


In [81]:
final_dataframe = final_output[0]
for i in range(1, len(final_output)):
    final_dataframe = final_dataframe.merge(final_output[i], on=["Dataset", "Language", "Metric", "Strategy"], how="outer")

In [82]:
final_dataframe.to_csv("results.csv", index=False)

In [83]:
final_dataframe.to_excel("results.xlsx", index=False)